In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
logging.basicConfig(level=logging.INFO)

KeyboardInterrupt: 

In [ ]:
seed_everything(0)

In [ ]:
use_contrastive = True
# use_contrastive = False
num_tasks = 5
num_epochs = 20


data_cfg = {
    "dataset_name": "mnist",
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
    # 'with_replacement': True,
}
dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


In [ ]:
seed_everything(7)
sender_dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [9 5 7 8 4 3 6 1 0 2]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


In [ ]:
net_cfg = {
    'depth': 4,
    'layer_size': 64,
    'num_init_tasks': num_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': num_tasks,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
}

In [ ]:
## MODULAR
NetCls = MLPSoftLLDynamic
LearnerCls = CompositionalDynamicER

## MONOLITHIC
NetCls = MLP
LearnerCls = NoComponentsER

sharing_cfg = DictConfig({
    "scorer": "cross_entropy",
    "num_queries": 4,
    "query_score_threshold": 0.0,
})
train_cfg = {
    # "num_epochs": 40,
    "num_epochs": num_epochs,
}

agent = RecvDataAgent(0, 0, dataset,
                NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, 
                sharing_cfg)

INFO:root:Agent: node_id: 0, seed: 0


In [ ]:
class ConcatenatedDataset(torch.utils.data.Dataset):
    def __init__(self, datasets):
        self.datasets = datasets

    def __len__(self):
        return sum(len(dataset) for dataset in self.datasets)
    
    def __getitem__(self, index):
        # Find the dataset which contains the data point with the given index
        for i, dataset in enumerate(self.datasets):
            if index < len(dataset):
                break
            index -= len(dataset)
            
        # Get the data from the dataset
        x, y = dataset[index]
        return x, y, i


def concatenate_datasets(sender_trainset):
    """
    Convert and concatenate sender_dataset.trainset into one big dataset.
    
    :param sender_trainset: List of datasets split by tasks.
    :return: Dataset with items (X, y_source, task_source_id).
    """
    
    return ConcatenatedDataset(sender_trainset)

In [ ]:
combined_data = concatenate_datasets(sender_dataset.trainset)
transformed_data = utilize_global_labels(combined_data, source_class_sequence=sender_dataset.class_sequence, target_class_sequence=
                                         dataset.class_sequence, num_classes_per_task=dataset.num_classes_per_task)

flipped_data = RandomFlippedDataset(transformed_data, flip_probability=1.0, num_classes=
                                    dataset.num_classes_per_task)

In [ ]:
# before additional data
for t in range(num_tasks):
    agent.train(t)

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.691	acc: 0.524
INFO:root:	task: avg	loss: 0.691	acc: 0.524
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.691	acc: 0.524
INFO:root:	task: avg	loss: 0.691	acc: 0.524
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.691	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: avg	loss: 0.693	acc: 0.508
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.691	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: avg	loss: 0.693	acc: 0.508
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.691	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: 2	loss: 0.700	acc: 0.481
INFO:root:	task: avg	loss: 0.695	acc: 0.499
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.691	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.493
INFO:root:	task: 2	loss: 0.700	acc: 0.481
INFO:root:	task: avg	loss: 0.695	acc: 0.499
INFO

In [ ]:
# mega_loader = torch.utils.data.DataLoader(transformed_data, batch_size=64, shuffle=True)
mega_loader = torch.utils.data.DataLoader(flipped_data, batch_size=64, shuffle=True)


task_id = num_tasks-1
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=256,
                                                         shuffle=False,
                                                         num_workers=4,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(agent.dataset.testset[:(task_id+1)])}


agent.agent._train(mega_loader, num_epochs=num_epochs,
                   task_id=task_id, testloaders=testloaders)

INFO:root:epochs: 1, training task: 4
INFO:root:	task: 0	loss: 0.684	acc: 0.524
INFO:root:	task: 1	loss: 0.680	acc: 0.803
INFO:root:	task: 2	loss: 0.671	acc: 0.481
INFO:root:	task: 3	loss: 0.679	acc: 0.799
INFO:root:	task: 4	loss: 0.686	acc: 0.787
INFO:root:	task: avg	loss: 0.680	acc: 0.679
INFO:root:epochs: 2, training task: 4
INFO:root:	task: 0	loss: 0.694	acc: 0.524
INFO:root:	task: 1	loss: 0.694	acc: 0.502
INFO:root:	task: 2	loss: 0.687	acc: 0.481
INFO:root:	task: 3	loss: 0.690	acc: 0.689
INFO:root:	task: 4	loss: 0.691	acc: 0.682
INFO:root:	task: avg	loss: 0.691	acc: 0.576
INFO:root:epochs: 3, training task: 4
INFO:root:	task: 0	loss: 0.696	acc: 0.524
INFO:root:	task: 1	loss: 0.696	acc: 0.387
INFO:root:	task: 2	loss: 0.690	acc: 0.481
INFO:root:	task: 3	loss: 0.693	acc: 0.546
INFO:root:	task: 4	loss: 0.692	acc: 0.609
INFO:root:	task: avg	loss: 0.693	acc: 0.509
INFO:root:epochs: 4, training task: 4
INFO:root:	task: 0	loss: 0.695	acc: 0.524
INFO:root:	task: 1	loss: 0.697	acc: 0.378
IN